In [ ]:
import mediapipe as mp
import cv2
import math
import os
import csv

mpDraw = mp.solutions.drawing_utils
mpPose = mp.solutions.pose
mp_holistic = mp.solutions.holistic
pose = mpPose.Pose()

In [ ]:
landmarks = ['class','left_elbow_angle','right_elbow_angle','left_shoulder_angle','right_shoulder_angle','left_knee_angle','right_knee_angle','left_hip_angle','right_hip_angle']

In [ ]:
with open('body_angles.csv', mode='w', newline='') as f:
    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)

In [ ]:
def calculateAngle(landmark1, landmark2, landmark3):
  

    # Get the required landmarks coordinates.
    x1, y1, _ = landmark1
    x2, y2, _ = landmark2
    x3, y3, _ = landmark3

    # Calculate the angle between the three points
    angle = math.degrees(math.atan2(y3 - y2, x3 - x2) - math.atan2(y1 - y2, x1 - x2))
    
    # Check if the angle is less than zero.
    if angle < 0:

        # Add 360 to the found angle.
        angle += 360
    
    # Return the calculated angle.
    return angle

In [ ]:
rootdir="dataset"
for file in os.listdir(rootdir):
    path=rootdir+'/'+file
    class_name=file
    for img in os.listdir(path):
        temp = []
        landmarks = []
        img = cv2.imread(path + "/" + img)
        height,width=img.shape[:2]
        # print(im_height," ",im_width)
        imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        results = pose.process(imgRGB)
        if results.pose_landmarks:
                for landmark in results.pose_landmarks.landmark:
                    landmarks.append((int(landmark.x * width), int(landmark.y * height),(landmark.z * width)))
                
                
                left_elbow_angle = calculateAngle(landmarks[mpPose.PoseLandmark.LEFT_SHOULDER.value],
                                    landmarks[mpPose.PoseLandmark.LEFT_ELBOW.value],
                                    landmarks[mpPose.PoseLandmark.LEFT_WRIST.value])

                # Get the angle between the right shoulder, elbow and wrist points. 
                right_elbow_angle = calculateAngle(landmarks[mpPose.PoseLandmark.RIGHT_SHOULDER.value],
                                                landmarks[mpPose.PoseLandmark.RIGHT_ELBOW.value],
                                                landmarks[mpPose.PoseLandmark.RIGHT_WRIST.value])   
                        
                # Get the angle between the left elbow, shoulder and hip points. 
                left_shoulder_angle = calculateAngle(landmarks[mpPose.PoseLandmark.LEFT_ELBOW.value],
                                                    landmarks[mpPose.PoseLandmark.LEFT_SHOULDER.value],
                                                    landmarks[mpPose.PoseLandmark.LEFT_HIP.value])

                # Get the angle between the right hip, shoulder and elbow points. 
                right_shoulder_angle = calculateAngle(landmarks[mpPose.PoseLandmark.RIGHT_HIP.value],
                                                    landmarks[mpPose.PoseLandmark.RIGHT_SHOULDER.value],
                                                    landmarks[mpPose.PoseLandmark.RIGHT_ELBOW.value])

                # Get the angle between the left hip, knee and ankle points. 
                left_knee_angle = calculateAngle(landmarks[mpPose.PoseLandmark.LEFT_HIP.value],
                                                landmarks[mpPose.PoseLandmark.LEFT_KNEE.value],
                                                landmarks[mpPose.PoseLandmark.LEFT_ANKLE.value])

                # Get the angle between the right hip, knee and ankle points 
                right_knee_angle = calculateAngle(landmarks[mpPose.PoseLandmark.RIGHT_HIP.value],
                                                landmarks[mpPose.PoseLandmark.RIGHT_KNEE.value],
                                                landmarks[mpPose.PoseLandmark.RIGHT_ANKLE.value])
                
                left_hip_angle=calculateAngle(landmarks[mpPose.PoseLandmark.LEFT_SHOULDER.value],
                                              landmarks[mpPose.PoseLandmark.LEFT_HIP.value],
                                              landmarks[mpPose.PoseLandmark.LEFT_KNEE.value])

                right_hip_angle=calculateAngle(landmarks[mpPose.PoseLandmark.RIGHT_SHOULDER.value],
                                              landmarks[mpPose.PoseLandmark.RIGHT_HIP.value],
                                              landmarks[mpPose.PoseLandmark.RIGHT_KNEE.value])
                
                temp = temp + [left_elbow_angle,right_elbow_angle,left_shoulder_angle,right_shoulder_angle,left_knee_angle,right_knee_angle,left_hip_angle,right_hip_angle]
                temp.insert(0, class_name)
                print(temp)
            
        # Export to CSV
                with open('body_angles.csv', mode='a', newline='') as f:
                        csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                        csv_writer.writerow(temp)